In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Parameters
img_height, img_width = 64, 64
batch_size = 32

# Data transformation and loading
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(root='homer_bart', transform=transform)
train_size = int(0.9 * len(train_dataset))
validation_size = len(train_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [train_size, validation_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)




In [102]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(64 * 64 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

model = SimpleNN()

In [103]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [104]:
epochs = 500
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if(epoch % 10 == 0):
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

Epoch 1, Loss: 0.6549168452620506
Epoch 11, Loss: 0.37563397362828255
Epoch 21, Loss: 0.3309764862060547
Epoch 31, Loss: 0.3256305530667305
Epoch 41, Loss: 0.32149936631321907
Epoch 51, Loss: 0.3212883360683918
Epoch 61, Loss: 0.32121486589312553
Epoch 71, Loss: 0.32116926461458206
Epoch 81, Loss: 0.32114364579319954
Epoch 91, Loss: 0.3211246244609356
Epoch 101, Loss: 0.32111189141869545
Epoch 111, Loss: 0.32410162314772606
Epoch 121, Loss: 0.3172062039375305
Epoch 131, Loss: 0.31719285249710083
Epoch 141, Loss: 0.3171866051852703
Epoch 151, Loss: 0.32022128626704216
Epoch 161, Loss: 0.3171800225973129
Epoch 171, Loss: 0.31717803701758385
Epoch 181, Loss: 0.3171762600541115
Epoch 191, Loss: 0.3171750456094742
Epoch 201, Loss: 0.31717410311102867
Epoch 211, Loss: 0.317173108458519
Epoch 221, Loss: 0.3171723000705242
Epoch 231, Loss: 0.317171897739172
Epoch 241, Loss: 0.3171713389456272
Epoch 251, Loss: 0.3171708732843399
Epoch 261, Loss: 0.3171706646680832
Epoch 271, Loss: 0.31717033684

In [105]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in validation_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 88.88888888888889%
